Requirements

* same function, same data, new behavior
* functions can call other functions (and behave like subclass method dispatch)
* easy for user to enter system from the outside
* does not require defining concrete methods over a data decorator class


## Single dispatch

In [1]:
from sqlalchemy import sql
from functools import singledispatch

In [2]:
@singledispatch
def mean(data):
    raise NotImplementedError()
    
@mean.register(sql.ClauseElement)
def _mean_sql_col(data):
    return sql.func.MEAN(data)

In [3]:
print(
    mean(sql.column('a'))
)

MEAN(a)


## Codata

In [4]:
# problem is if we want this function to do different things
# in certain situations
def mean2(data):
    return sql.func.AVG(data)

mean2(sql.column('a'))

<sqlalchemy.sql.functions.Function at 0x10c07fe80; AVG>

In [5]:
# Note ClauseElement is the base of most sqlalchemy classes
class PostgresMethods(sql.ClauseElement): pass

class DataDecorator():
    def __init__(self, data, eliminator_cls):
        self.data = data
        self.eliminator_cls = eliminator_cls
    
    def apply(self, dispatcher, args = (), kwargs = {}):
        method = dispatcher.dispatch(self.eliminator_cls)
        return method(self.data, *args, **kwargs)
        
    def wrap(self, res):
        return self.__class__(res, self.eliminator_cls)
    
    def visit(self, dispatcher, args = (), kwargs = {}):
        res = self.apply(dispatcher, args, kwargs)
        return self.wrap(res)

In [6]:
dd = DataDecorator(sql.column('a'), PostgresMethods)

dd.visit(mean)


In [7]:
@mean.register(PostgresMethods)
def _(data):
    return sql.func.AVG(data)


@mean.register(DataDecorator)
def _(data):
    return data.visit(mean)

print(mean(mean(dd)).data)

AVG(AVG(a))


### dispatch on DataDecorator to use methods in methods

In [8]:
@singledispatch
def new_func(x): raise NotImplementedError()
    
@new_func.register(DataDecorator)
def _(x):
    print("Calling mean")
    dd_avg = mean(x)
    
    return dd.wrap(x.data + dd_avg.data)

print(new_func(dd).data)

Calling mean
a + AVG(a)
